In [17]:
import nibabel as nib
from skimage.transform import resize
import numpy as np
import os
import glob
from joblib import Parallel, delayed

In [ ]:
fastsurfer_vinn_dir = '/home/ibrazug/Dokumente/KindersegV2/Ibra/derivatives/FastSurferVINN'
freesurfer6_dir = '/home/ibrazug/Dokumente/KindersegV2/Ibra/derivatives/FREESURFER_SEG'
freesurfer6_resized_dir = '/home/ibrazug/Dokumente/KindersegV2/Ibra/derivatives/Freesurfer6_resized'
sids = [x for x in os.listdir(fastsurfer_vinn_dir) if x.startswith('sub-')]
len(sids)

In [19]:
mask_labels = [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]

In [20]:
def resize_mask(mask, new_shape):
    return resize(mask, new_shape, order=0, mode='constant', anti_aliasing=False, preserve_range=True)

def dice_score(mask1, mask2, n=1):
    mask1 = np.where(mask1 == n, 1, 0).astype(bool)
    mask2 = np.where(mask2 == n, 1, 0).astype(bool)
    intersection = np.sum(mask1 & mask2)
    return 2 * intersection / (np.sum(mask1) + np.sum(mask2))

In [ ]:
dscs = dict()
for sid in sids:
    try:
        fastsurfer_mask = glob.glob(f'{fastsurfer_vinn_dir}/{sid}/mri/aparc.DKTatlas+aseg.deep.mgz')[0]
        freesurfer_mask = glob.glob(f'{freesurfer6_dir}/{sid}.reconall/mri/aparc.DKTatlas+aseg.mgz')[0]
    except:
        print(f'No mask found for {sid}')
        continue
    freesurfer_mask_nii = nib.load(freesurfer_mask)
    freesurfer_mask_data = freesurfer_mask_nii.get_fdata().round(0).astype(np.uint8)
    fastsurfer_mask = nib.load(fastsurfer_mask)
    fastsurfer_mask_data = fastsurfer_mask.get_fdata().round(0).astype(np.uint8)
    fastsurfer_mask_shape = fastsurfer_mask_data.shape
    freesurfer_mask_data_resized = resize_mask(freesurfer_mask_data, fastsurfer_mask_shape)
    dscs[sid] = Parallel(n_jobs=16)(delayed(dice_score)(fastsurfer_mask_data, freesurfer_mask_data_resized, n) for n in mask_labels)
    freesurfer_mask_resized_nii = nib.Nifti1Image(freesurfer_mask_data_resized, fastsurfer_mask.affine, fastsurfer_mask.header)
    nib.save(freesurfer_mask_resized_nii, f'{freesurfer6_resized_dir}/{sid}.nii.gz')


In [ ]:
fastsurfer_resized_dir = '/home/ibrazug/Dokumente/KindersegV2/Ibra/derivatives/FastSurferVINN_resized'
dscs2 = dict()
for sid in sids:
    try:
        fastsurfer_mask = glob.glob(f'{fastsurfer_vinn_dir}/{sid}/mri/aparc.DKTatlas+aseg.deep.mgz')[0]
        freesurfer_mask = glob.glob(f'{freesurfer6_dir}/{sid}.reconall/mri/aparc.DKTatlas+aseg.mgz')[0]
    except:
        print(f'No mask found for {sid}')
        continue
    freesurfer_mask_nii = nib.load(freesurfer_mask)
    freesurfer_mask_data = freesurfer_mask_nii.get_fdata().round(0).astype(np.uint8)
    fastsurfer_mask = nib.load(fastsurfer_mask)
    fastsurfer_mask_data = fastsurfer_mask.get_fdata().round(0).astype(np.uint8)
    freesurfer_mask_shape = freesurfer_mask_data.shape
    fastsurfer_mask_data_resized = resize_mask(fastsurfer_mask_data, freesurfer_mask_shape)
    dscs2[sid] = Parallel(n_jobs=16)(delayed(dice_score)(fastsurfer_mask_data_resized, freesurfer_mask_data, n) for n in mask_labels)
    fastsurfer_mask_resized_nii = nib.Nifti1Image(fastsurfer_mask_data_resized, freesurfer_mask_nii.affine, freesurfer_mask_nii.header)
    nib.save(fastsurfer_mask_resized_nii, f'{fastsurfer_resized_dir}/{sid}.nii.gz')


In [ ]:
import pandas as pd

In [ ]:
dscs_df2 = pd.DataFrame(dscs).T
dscs_df2.columns = mask_labels
dscs_df2['class'] = 'freesurfer_resized'

dscs_df = pd.DataFrame(dscs2).T
dscs_df.columns = mask_labels
dscs_df['class'] = 'fastsurfer_resized'

dscs_df = pd.concat([dscs_df, dscs_df2])

In [ ]:
dscs_df.head()

In [32]:
dscs_df.to_csv('/home/ibrazug/Dokumente/KindersegV2/review_data/dscs.csv', index=True)